In [61]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import tqdm

%run ../Scripts/trajectory.py
%run ../Scripts/trajectoryVisual.py
%run ../Scripts/trajsTree.py
%run ../Scripts/supMethods.py
foldername = "../Data/test1"

In [6]:
with open(foldername + "/params.json") as json_file:
    params = json.load(json_file)
with open(foldername + "/sim_params.json") as json_file:
    sim_params = json.load(json_file)

In [ ]:
i = 700
n_i = scipy.sparse.load_npz(foldername+f"/sp_frame_n{i}.npz").todok()
indexes = get_nonzero_w_repeats(n_i)
means_gmm, covs_gmm, counts_gmm = fit_unknown_GMM(indexes, w = 1000)
means, covs, counts = reduce_GMM(means_gmm, covs_gmm, counts_gmm)
plot_Ellipses(n_i, i, means, covs, save = False, foldername = foldername)

In [28]:
t0 = 300
tf = sim_params["tf"]
dt = 25
t_domain = np.arange(t0, tf, dt)

In [200]:
for t in t_domain:
    n_i = scipy.sparse.load_npz(foldername+f"/sp_frame_n{t}.npz").todok()
    indexes = get_nonzero_w_repeats(n_i)
    means_gmm, covs_gmm, counts_gmm = fit_unknown_GMM(indexes, w = 1000)
    means, covs, counts = reduce_GMM(means_gmm, covs_gmm, counts_gmm)
    plot_Ellipses(n_i, t, means, covs, save = True,
                foldername = foldername, input_color = "teal")
    
make_Gif(foldername, t_domain, typename = "GMM_plots")

In [63]:
for t in t_domain:
    n_i = scipy.sparse.load_npz(foldername+f"/sp_frame_n{t}.npz").todok()
    indexes = get_nonzero_w_repeats(n_i)
    means_gmm, covs_gmm, counts_gmm = fit_unknown_GMM(indexes, w = 1000)
    means, covs, counts = reduce_GMM(means_gmm, covs_gmm, counts_gmm)
    next_list = make_Treelist(t, means, covs, counts)
    
    if t == t0:
        init_list = next_list
        prev_list = next_list
        continue

    prev_list = link_Treelists(prev_list, next_list)
    prev_list = next_list

save_Treelist(foldername, init_list)

In [ ]:
colors = generate_colors(len(init_list))
plt.figure()
plt.title(f"Trajectories starting from {t0} to {tf}")

for root_node, color in zip(init_list, colors):
    for trajs in root_node.get_all_traversals:
        positions = [x.mean for x in trajs]
        x_val, y_val = extract_xy(positions)
        plt.plot(x_val, y_val, color = color)
